# Part 3: Advanced Analysis

In this part, we will implement advanced analysis techniques for physiological time series data, including time-domain feature extraction, frequency analysis, and wavelet transforms.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
import pywt

sns.set()
sns.set_context('notebook')

## 1. Time-Domain Feature Extraction

Implement the `extract_time_domain_features` function to extract various time-domain features from physiological signals.

In [ ]:
import pandas as pd
import numpy as np

def extract_time_domain_features(data, window_size=60):
    window_samples = window_size
    features = pd.DataFrame(index=data.index)

    features['mean'] = data['heart_rate'].rolling(window=window_samples, min_periods=1).mean()
    features['std'] = data['heart_rate'].rolling(window=window_samples, min_periods=1).std()
    features['min'] = data['heart_rate'].rolling(window=window_samples, min_periods=1).min()
    features['max'] = data['heart_rate'].rolling(window=window_samples, min_periods=1).max()

    features['mean_hr'] = data['heart_rate'].rolling(window=window_samples, min_periods=1).mean()
    features['std_hr'] = data['heart_rate'].rolling(window=window_samples, min_periods=1).std()

    diff_hr = data['heart_rate'].diff()
    diff_sq = diff_hr.pow(2)
    features['rmssd'] = diff_sq.rolling(window=window_samples, min_periods=1).mean().apply(np.sqrt)
    features['sdnn'] = data['heart_rate'].rolling(window=window_samples, min_periods=1).std()
    diff_gt_50 = (diff_hr.abs() > 50).astype(int)
    features['pnn50'] = diff_gt_50.rolling(window=window_samples, min_periods=1).mean() * 100

    features = features.fillna(0.0)

    return features


## 2. Frequency Analysis

Implement the `analyze_frequency_components` function to perform frequency-domain analysis on the signals.

In [10]:
import numpy as np
import pandas as pd
from scipy import signal
from pathlib import Path
import os

def analyze_frequency_components(data, sampling_rate, window_size=60, output_dir='frequency_analysis', filename_prefix='fft_results'):
    """Perform frequency-domain analysis on physiological signals.

    Parameters
    ----------
    data : pandas.DataFrame
        Input data with columns: ['timestamp', 'heart_rate', 'eda', 'temperature', 'subject_id', 'session'].
    sampling_rate : float
        Sampling rate of the signal in Hz.
    window_size : int, optional
        Size of the analysis window in seconds, default=60.
    output_dir : str
        Directory to save the frequency analysis results.
    filename_prefix : str
        Prefix for the saved files.

    Returns
    -------
    dict
        Dictionary containing frequency analysis results:
        - 'frequencies': array of frequency values
        - 'power': array of power spectrum values
        - 'bands': dictionary of band powers
    """
    # Create output directory
    output_dir = Path(output_dir).expanduser()
    output_dir.mkdir(parents=True, exist_ok=True)

    # Convert window_size from seconds to number of samples
    window_samples = int(window_size * sampling_rate)

    # Initialize results
    all_frequencies = []
    all_power = []

    n_windows = len(data) // window_samples

    for i in range(n_windows):
        window_data = data['heart_rate'].iloc[i*window_samples:(i+1)*window_samples]

        # Calculate PSD using Welch's method
        frequencies, power = signal.welch(
            window_data,
            fs=sampling_rate,
            nperseg=window_samples
        )

        all_frequencies.append(frequencies)
        all_power.append(power)

    # Average across windows
    avg_frequencies = np.mean(all_frequencies, axis=0)
    avg_power = np.mean(all_power, axis=0)

    # Define frequency bands
    bands = {
        'VLF': (0.003, 0.04),
        'LF': (0.04, 0.15),
        'HF': (0.15, 0.4)
    }

    band_powers = {}
    for band_name, (low, high) in bands.items():
        mask = (avg_frequencies >= low) & (avg_frequencies <= high)
        band_powers[band_name] = np.trapz(avg_power[mask], avg_frequencies[mask])  # Integrate PSD over the band

    # LF/HF ratio
    if band_powers['HF'] > 0:
        band_powers['LF/HF'] = band_powers['LF'] / band_powers['HF']
    else:
        band_powers['LF/HF'] = np.nan  # Avoid division by zero

    results = {
        'frequencies': avg_frequencies,
        'power': avg_power,
        'bands': band_powers
    }

    # Save results
    np.savez(output_dir / f"{filename_prefix}.npz", frequencies=avg_frequencies, power=avg_power, bands=band_powers)
    np.save(output_dir / f"{filename_prefix}_power.npy", avg_power)
    pd.DataFrame({
        'frequency_hz': avg_frequencies,
        'power_density': avg_power
    }).to_csv(output_dir / f"{filename_prefix}_power.csv", index=False)

    return results


## 3. Time-Frequency Analysis

Implement the `analyze_time_frequency_features` function to analyze time-frequency features using wavelet transforms.

In [11]:
import numpy as np
import pandas as pd
import pywt
from pathlib import Path
import os

def analyze_time_frequency_features(data, sampling_rate, window_size=60, output_dir='time_frequency_analysis', filename_prefix='wavelet_results'):
    """Analyze time-frequency features using wavelet transforms.

    Parameters
    ----------
    data : pandas.DataFrame
        Input data with columns: ['timestamp', 'heart_rate', 'eda', 'temperature', 'subject_id', 'session'].
    sampling_rate : float
        Sampling rate of the signal in Hz.
    window_size : int, optional
        Size of the analysis window in seconds, default=60.
    output_dir : str
        Directory to save time-frequency analysis results.
    filename_prefix : str
        Prefix for the saved files.

    Returns
    -------
    dict
        Dictionary containing:
            - 'scales': array of wavelet scales
            - 'coefficients': array of wavelet coefficients
            - 'time_frequency_energy': 2D array of energy distribution
            - 'dominant_frequencies': array of dominant frequency per time point
    """
    # Create output directory
    output_dir = Path(output_dir).expanduser()
    output_dir.mkdir(parents=True, exist_ok=True)

    # Convert window_size from seconds to number of samples
    window_samples = int(window_size * sampling_rate)

    # Define wavelet scales
    scales = np.arange(1, 128)

    # Initialize
    all_coefficients = []
    all_energy = []
    all_dominant_freqs = []

    n_windows = len(data) // window_samples

    for i in range(n_windows):
        window_data = data['heart_rate'].iloc[i*window_samples:(i+1)*window_samples]

        # Apply continuous wavelet transform
        coefficients, frequencies = pywt.cwt(
            window_data,
            scales,
            'morl',
            sampling_period=1/sampling_rate
        )

        energy = np.abs(coefficients) ** 2  # Time-frequency energy

        # Dominant frequency over time (maximum energy at each time step)
        dominant_freq_indices = np.argmax(energy, axis=0)
        dominant_freqs = frequencies[dominant_freq_indices]

        all_coefficients.append(coefficients)
        all_energy.append(energy)
        all_dominant_freqs.append(dominant_freqs)

    # Average across windows
    avg_coefficients = np.mean(all_coefficients, axis=0)
    avg_energy = np.mean(all_energy, axis=0)
    avg_dominant_freqs = np.mean(all_dominant_freqs, axis=0)

    results = {
        'scales': scales,
        'coefficients': avg_coefficients,
        'time_frequency_energy': avg_energy,
        'dominant_frequencies': avg_dominant_freqs
    }

    # Save results
    np.savez(output_dir / f"{filename_prefix}.npz", 
             scales=scales, 
             coefficients=avg_coefficients, 
             time_frequency_energy=avg_energy, 
             dominant_frequencies=avg_dominant_freqs)

    np.save(output_dir / f"{filename_prefix}_energy.npy", avg_energy)
    
    pd.DataFrame({
        'time_point': np.arange(len(avg_dominant_freqs)),
        'dominant_frequency_hz': avg_dominant_freqs
    }).to_csv(output_dir / f"{filename_prefix}_dominant_freq.csv", index=False)

    return results


## Example Usage

Here's how to use these functions with your data:

In [13]:
# Load your data
data = pd.read_csv('~/Documents/4-it-s-about-time-kanting6/data/S1_processed.csv')

# Extract time-domain features
features = extract_time_domain_features(data, window_size=60)
print("Time-domain features:")
print(features.head())

# Analyze frequency components
sampling_rate = 4.0  # Hz
freq_results = analyze_frequency_components(data, sampling_rate, window_size=60)
print("\nFrequency analysis results:")
print("Frequency bands:", freq_results['bands'])

# Analyze time-frequency features
tf_results = analyze_time_frequency_features(data, sampling_rate, window_size=60)
print("\nTime-frequency analysis results:")
print("Wavelet scales:", tf_results['scales'].shape)
print("Coefficients shape:", tf_results['coefficients'].shape)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kankantingting/Documents/4-it-s-about-time-kanting6/data/S1_processed.csv'